In [1]:
import pandas as pd
import numpy as np

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Setting up variable values as column

#### Using stack method

In [3]:
state_fruit = pd.read_csv('data/state_fruit.csv', index_col=0)
state_fruit

Apple  Orange  Banana
Texas       12      10      40
Arizona      9       7      12
Florida      0      14     190

#### column names are not variables, rather than values
#### example, fruit should be column name and Apple should be value.
#### Also we dont know what are those values means, weight?, sales?

In [4]:
state_fruit.stack()  ## stacks columns to index

Texas    Apple      12
         Orange     10
         Banana     40
Arizona  Apple       9
         Orange      7
         Banana     12
Florida  Apple       0
         Orange     14
         Banana    190
dtype: int64

#### Change from multi index to index

In [5]:
state_fruit_tidy = state_fruit.stack().reset_index()
state_fruit_tidy

level_0 level_1    0
0    Texas   Apple   12
1    Texas  Orange   10
2    Texas  Banana   40
3  Arizona   Apple    9
4  Arizona  Orange    7
5  Arizona  Banana   12
6  Florida   Apple    0
7  Florida  Orange   14
8  Florida  Banana  190

In [6]:
state_fruit_tidy.columns = ['state', 'fruit', 'weight']
state_fruit_tidy

state   fruit  weight
0    Texas   Apple      12
1    Texas  Orange      10
2    Texas  Banana      40
3  Arizona   Apple       9
4  Arizona  Orange       7
5  Arizona  Banana      12
6  Florida   Apple       0
7  Florida  Orange      14
8  Florida  Banana     190

#### Other way of doing the same

In [7]:
state_fruit.stack().rename_axis(['state', 'fruit'])

state    fruit 
Texas    Apple      12
         Orange     10
         Banana     40
Arizona  Apple       9
         Orange      7
         Banana     12
Florida  Apple       0
         Orange     14
         Banana    190
dtype: int64

In [8]:
state_fruit.stack().rename_axis(['state', 'fruit']).reset_index(name='weight')

state   fruit  weight
0    Texas   Apple      12
1    Texas  Orange      10
2    Texas  Banana      40
3  Arizona   Apple       9
4  Arizona  Orange       7
5  Arizona  Banana      12
6  Florida   Apple       0
7  Florida  Orange      14
8  Florida  Banana     190

#### Using melt method

In [9]:
state_fruit_2 = pd.read_csv('data/state_fruit2.csv')
state_fruit_2

State  Apple  Orange  Banana
0    Texas     12      10      40
1  Arizona      9       7      12
2  Florida      0      14     190

In [10]:
state_fruit_2.melt(id_vars=['State'], value_vars=['Apple', 'Orange', 'Banana'])

State variable  value
0    Texas    Apple     12
1  Arizona    Apple      9
2  Florida    Apple      0
3    Texas   Orange     10
4  Arizona   Orange      7
5  Florida   Orange     14
6    Texas   Banana     40
7  Arizona   Banana     12
8  Florida   Banana    190

In [11]:
state_fruit_2.melt(id_vars=['State'],
                    value_vars=['Apple', 'Orange', 'Banana'],
                      var_name='Fruit',
                      value_name='Weight')

State   Fruit  Weight
0    Texas   Apple      12
1  Arizona   Apple       9
2  Florida   Apple       0
3    Texas  Orange      10
4  Arizona  Orange       7
5  Florida  Orange      14
6    Texas  Banana      40
7  Arizona  Banana      12
8  Florida  Banana     190

In [12]:
x = state_fruit_2.melt(id_vars='State')  ## variables names can be taken default from column names
x.columns = ['State', 'Fruit', 'Weight']
x

State   Fruit  Weight
0    Texas   Apple      12
1  Arizona   Apple       9
2  Florida   Apple       0
3    Texas  Orange      10
4  Arizona  Orange       7
5  Florida  Orange      14
6    Texas  Banana      40
7  Arizona  Banana      12
8  Florida  Banana     190

### stacking multiple columns

In [13]:
movie = pd.read_csv('data/movie.csv')

In [14]:
actor_columns = ['movie_title', 'actor_1_name',
                'actor_2_name', 'actor_3_name',
                'actor_1_facebook_likes',
                'actor_2_facebook_likes',
                'actor_3_facebook_likes']

In [15]:
actor = movie[actor_columns]

In [16]:
actor.head(2)

movie_title actor_1_name      actor_2_name  \
0                                    Avatar  CCH Pounder  Joel David Moore   
1  Pirates of the Caribbean: At World's End  Johnny Depp     Orlando Bloom   

     actor_3_name  actor_1_facebook_likes  actor_2_facebook_likes  \
0       Wes Studi                  1000.0                   936.0   
1  Jack Davenport                 40000.0                  5000.0   

   actor_3_facebook_likes  
0                   855.0  
1                  1000.0

In [17]:
actor.columns

Index(['movie_title', 'actor_1_name', 'actor_2_name', 'actor_3_name',
       'actor_1_facebook_likes', 'actor_2_facebook_likes',
       'actor_3_facebook_likes'],
      dtype='object')

In [18]:
def change_col_name(col_name):
    col_name = col_name.replace('_name', '')
    if 'facebook' in col_name:
        fb_idx = col_name.find('facebook')
        col_name = col_name[:5] + col_name[fb_idx - 1:] \
                                + col_name[5:fb_idx-1]
    return col_name

In [19]:
actor2 = actor.rename(columns=change_col_name)
actor2.head()

movie_title          actor_1  \
0                                      Avatar      CCH Pounder   
1    Pirates of the Caribbean: At World's End      Johnny Depp   
2                                     Spectre  Christoph Waltz   
3                       The Dark Knight Rises        Tom Hardy   
4  Star Wars: Episode VII - The Force Awakens      Doug Walker   

            actor_2               actor_3  actor_facebook_likes_1  \
0  Joel David Moore             Wes Studi                  1000.0   
1     Orlando Bloom        Jack Davenport                 40000.0   
2      Rory Kinnear      Stephanie Sigman                 11000.0   
3    Christian Bale  Joseph Gordon-Levitt                 27000.0   
4        Rob Walker                   NaN                   131.0   

   actor_facebook_likes_2  actor_facebook_likes_3  
0                   936.0                   855.0  
1                  5000.0                  1000.0  
2                   393.0                   161.0  
3                 23000.0                 23000.0  
4                    12.0                     NaN

In [20]:
stubs = ['actor', 'actor_facebook_likes']
actor2_tidy = pd.wide_to_long(actor2, 
                              stubnames=stubs, 
                              i=['movie_title'], 
                              j='actor_num', 
                              sep='_')
actor2_tidy.head()

actor  \
movie_title                                actor_num                    
Avatar                                     1              CCH Pounder   
Pirates of the Caribbean: At World's End   1              Johnny Depp   
Spectre                                    1          Christoph Waltz   
The Dark Knight Rises                      1                Tom Hardy   
Star Wars: Episode VII - The Force Awakens 1              Doug Walker   

                                                      actor_facebook_likes  
movie_title                                actor_num                        
Avatar                                     1                        1000.0  
Pirates of the Caribbean: At World's End   1                       40000.0  
Spectre                                    1                       11000.0  
The Dark Knight Rises                      1                       27000.0  
Star Wars: Episode VII - The Force Awakens 1                         131.0

In [21]:
df = pd.read_csv('data/stackme.csv')
df

State Country    a1   b2   Test  d  e
0    TX      US  0.45  0.3  Test1  2  6
1    MA      US  0.03  1.2  Test2  9  7
2    ON     CAN  0.70  4.2  Test3  4  2

In [22]:
df2 = df.rename(columns = {'a1':'group1_a1', 'b2':'group1_b2',
                               'd':'group2_a1', 'e':'group2_b2'})
df2

State Country  group1_a1  group1_b2   Test  group2_a1  group2_b2
0    TX      US       0.45        0.3  Test1          2          6
1    MA      US       0.03        1.2  Test2          9          7
2    ON     CAN       0.70        4.2  Test3          4          2

In [23]:
pd.wide_to_long(df2, 
                    stubnames=['group1', 'group2'], 
                    i=['State', 'Country', 'Test'], 
                    j='Label', 
                    suffix='.+', 
                    sep='_')

group1  group2
State Country Test  Label                
TX    US      Test1 a1       0.45       2
                    b2       0.30       6
MA    US      Test2 a1       0.03       9
                    b2       1.20       7
ON    CAN     Test3 a1       0.70       4
                    b2       4.20       2

### Invert of the stacking, (values to variables)

In [24]:
usecol_func = lambda x: 'UGDS_' in x or x == 'INSTNM'

In [25]:
college = pd.read_csv('data/college.csv',
                      index_col='INSTNM',
                     usecols = usecol_func)

In [26]:
college.head(3)

UGDS_WHITE  UGDS_BLACK  UGDS_HISP  \
INSTNM                                                                   
Alabama A & M University                 0.0333      0.9353     0.0055   
University of Alabama at Birmingham      0.5922      0.2600     0.0283   
Amridge University                       0.2990      0.4192     0.0069   

                                     UGDS_ASIAN  UGDS_AIAN  UGDS_NHPI  \
INSTNM                                                                  
Alabama A & M University                 0.0019     0.0024     0.0019   
University of Alabama at Birmingham      0.0518     0.0022     0.0007   
Amridge University                       0.0034     0.0000     0.0000   

                                     UGDS_2MOR  UGDS_NRA  UGDS_UNKN  
INSTNM                                                               
Alabama A & M University                0.0000    0.0059     0.0138  
University of Alabama at Birmingham     0.0368    0.0179     0.0100  
Amridge University                      0.0000    0.0000     0.2715

In [27]:
college_stack = college.stack()   # default stacking, all columns

In [28]:
college_stack.head(18)

INSTNM                                         
Alabama A & M University             UGDS_WHITE    0.0333
                                     UGDS_BLACK    0.9353
                                     UGDS_HISP     0.0055
                                     UGDS_ASIAN    0.0019
                                     UGDS_AIAN     0.0024
                                     UGDS_NHPI     0.0019
                                     UGDS_2MOR     0.0000
                                     UGDS_NRA      0.0059
                                     UGDS_UNKN     0.0138
University of Alabama at Birmingham  UGDS_WHITE    0.5922
                                     UGDS_BLACK    0.2600
                                     UGDS_HISP     0.0283
                                     UGDS_ASIAN    0.0518
                                     UGDS_AIAN     0.0022
                                     UGDS_NHPI     0.0007
                                     UGDS_2MOR     0.0368
                        

In [29]:
college_unstacked = college_stack.unstack()

In [30]:
college_unstacked.head(3)

UGDS_WHITE  UGDS_BLACK  UGDS_HISP  \
INSTNM                                                                   
Alabama A & M University                 0.0333      0.9353     0.0055   
University of Alabama at Birmingham      0.5922      0.2600     0.0283   
Amridge University                       0.2990      0.4192     0.0069   

                                     UGDS_ASIAN  UGDS_AIAN  UGDS_NHPI  \
INSTNM                                                                  
Alabama A & M University                 0.0019     0.0024     0.0019   
University of Alabama at Birmingham      0.0518     0.0022     0.0007   
Amridge University                       0.0034     0.0000     0.0000   

                                     UGDS_2MOR  UGDS_NRA  UGDS_UNKN  
INSTNM                                                               
Alabama A & M University                0.0000    0.0059     0.0138  
University of Alabama at Birmingham     0.0368    0.0179     0.0100  
Amridge University                      0.0000    0.0000     0.2715

In [31]:
college2 = pd.read_csv('data/college.csv', 
                          usecols=usecol_func)
college2.head(3)

INSTNM  UGDS_WHITE  UGDS_BLACK  UGDS_HISP  \
0             Alabama A & M University      0.0333      0.9353     0.0055   
1  University of Alabama at Birmingham      0.5922      0.2600     0.0283   
2                   Amridge University      0.2990      0.4192     0.0069   

   UGDS_ASIAN  UGDS_AIAN  UGDS_NHPI  UGDS_2MOR  UGDS_NRA  UGDS_UNKN  
0      0.0019     0.0024     0.0019     0.0000    0.0059     0.0138  
1      0.0518     0.0022     0.0007     0.0368    0.0179     0.0100  
2      0.0034     0.0000     0.0000     0.0000    0.0000     0.2715

In [32]:
college_melted = college2.melt(id_vars='INSTNM',
                            var_name='Race',
                             value_name='Percentage')
college_melted.head(4)

INSTNM        Race  Percentage
0             Alabama A & M University  UGDS_WHITE      0.0333
1  University of Alabama at Birmingham  UGDS_WHITE      0.5922
2                   Amridge University  UGDS_WHITE      0.2990
3  University of Alabama in Huntsville  UGDS_WHITE      0.6988

In [33]:
melted_inv = college_melted.pivot(index='INSTNM', 
                                      columns='Race',
                                      values='Percentage')
melted_inv.head(3)

Race                                     UGDS_2MOR  UGDS_AIAN  UGDS_ASIAN  \
INSTNM                                                                      
A & W Healthcare Educators                     0.0        0.0      0.0000   
A T Still University of Health Sciences        NaN        NaN         NaN   
ABC Beauty Academy                             0.0        0.0      0.9333   

Race                                     UGDS_BLACK  UGDS_HISP  UGDS_NHPI  \
INSTNM                                                                      
A & W Healthcare Educators                   0.9750     0.0250        0.0   
A T Still University of Health Sciences         NaN        NaN        NaN   
ABC Beauty Academy                           0.0333     0.0333        0.0   

Race                                     UGDS_NRA  UGDS_UNKN  UGDS_WHITE  
INSTNM                                                                    
A & W Healthcare Educators                    0.0        0.0         0.0  
A T Still University of Health Sciences       NaN        NaN         NaN  
ABC Beauty Academy                            0.0        0.0         0.0

In [34]:
college2_replication = melted_inv.loc[college2['INSTNM'],
                                    college2.columns[1:]]\
                                     .reset_index()

In [35]:
college2_replication.head(3)

INSTNM  UGDS_WHITE  UGDS_BLACK  UGDS_HISP  \
0             Alabama A & M University      0.0333      0.9353     0.0055   
1  University of Alabama at Birmingham      0.5922      0.2600     0.0283   
2                   Amridge University      0.2990      0.4192     0.0069   

   UGDS_ASIAN  UGDS_AIAN  UGDS_NHPI  UGDS_2MOR  UGDS_NRA  UGDS_UNKN  
0      0.0019     0.0024     0.0019     0.0000    0.0059     0.0138  
1      0.0518     0.0022     0.0007     0.0368    0.0179     0.0100  
2      0.0034     0.0000     0.0000     0.0000    0.0000     0.2715

In [36]:
college2.equals(college2_replication)

True

### Unstacking after aggregation of data frame

In [37]:
employee = pd.read_csv('data/employee.csv')

In [38]:
(employee.groupby('RACE')['BASE_SALARY'] 
        .mean() 
        .astype(int)
        .sort_values(ascending=False))

RACE
White                                64419
Asian/Pacific Islander               61660
American Indian or Alaskan Native    60272
Hispanic/Latino                      52345
Others                               51278
Black or African American            50137
Name: BASE_SALARY, dtype: int64

In [39]:
agg = (employee.groupby(['RACE', 'GENDER'])['BASE_SALARY'] 
                .mean() 
                .astype(int)
                .sort_values(ascending=False))

In [40]:
agg

RACE                               GENDER
White                              Female    66793
                                   Male      63940
Others                             Female    63785
Asian/Pacific Islander             Female    63226
                                   Male      61033
American Indian or Alaskan Native  Male      60305
                                   Female    60238
Hispanic/Latino                    Male      54782
Black or African American          Male      51082
                                   Female    48915
Hispanic/Latino                    Female    46503
Others                             Male      38771
Name: BASE_SALARY, dtype: int64

In [41]:
agg.unstack('GENDER')

GENDER                             Female   Male
RACE                                            
American Indian or Alaskan Native   60238  60305
Asian/Pacific Islander              63226  61033
Black or African American           48915  51082
Hispanic/Latino                     46503  54782
Others                              63785  38771
White                               66793  63940

In [42]:
agg.unstack('RACE')

RACE    American Indian or Alaskan Native  Asian/Pacific Islander  \
GENDER                                                              
Female                              60238                   63226   
Male                                60305                   61033   

RACE    Black or African American  Hispanic/Latino  Others  White  
GENDER                                                             
Female                      48915            46503   63785  66793  
Male                        51082            54782   38771  63940

In [43]:
agg2 = employee.groupby(['RACE', 'GENDER'])['BASE_SALARY'] \
                   .agg(['mean', 'max', 'min']).astype(int)
agg2

mean     max    min
RACE                              GENDER                      
American Indian or Alaskan Native Female  60238   98536  26125
                                  Male    60305   81239  26125
Asian/Pacific Islander            Female  63226  130416  26125
                                  Male    61033  163228  27914
Black or African American         Female  48915  150416  24960
                                  Male    51082  275000  26125
Hispanic/Latino                   Female  46503  126115  26125
                                  Male    54782  165216  26104
Others                            Female  63785   63785  63785
                                  Male    38771   38771  38771
White                             Female  66793  178331  27955
                                  Male    63940  210588  26125

In [44]:
agg2.unstack('GENDER')

mean            max            min       
GENDER                            Female   Male  Female    Male Female   Male
RACE                                                                         
American Indian or Alaskan Native  60238  60305   98536   81239  26125  26125
Asian/Pacific Islander             63226  61033  130416  163228  26125  27914
Black or African American          48915  51082  150416  275000  24960  26125
Hispanic/Latino                    46503  54782  126115  165216  26125  26104
Others                             63785  38771   63785   38771  63785  38771
White                              66793  63940  178331  210588  27955  26125

### using groupby to get the functionlaity as pivot_table

In [45]:
flights = pd.read_csv('data/flights.csv')

In [46]:
flights.head(2)

MONTH  DAY  WEEKDAY AIRLINE ORG_AIR DEST_AIR  SCHED_DEP  DEP_DELAY  \
0      1    1        4      WN     LAX      SLC       1625       58.0   
1      1    1        4      UA     DEN      IAD        823        7.0   

   AIR_TIME  DIST  SCHED_ARR  ARR_DELAY  DIVERTED  CANCELLED  
0      94.0   590       1905       65.0         0          0  
1     154.0  1452       1333      -13.0         0          0

In [47]:
fp = flights.pivot_table(index='AIRLINE', 
                         columns='ORG_AIR', 
                         values='CANCELLED', 
                         aggfunc='sum',
                         fill_value=0).round(2)

In [48]:
fp.head(5)

ORG_AIR  ATL  DEN  DFW  IAH  LAS  LAX  MSP  ORD  PHX  SFO
AIRLINE                                                  
AA         3    4   86    3    3   11    3   35    4    2
AS         0    0    0    0    0    0    0    0    0    0
B6         0    0    0    0    0    0    0    0    0    1
DL        28    1    0    0    1    1    4    0    1    2
EV        18    6   27   36    0    0    6   53    0    0

In [49]:
fg = flights.groupby(['AIRLINE', 'ORG_AIR'])['CANCELLED'].sum()
fg.head()

AIRLINE  ORG_AIR
AA       ATL         3
         DEN         4
         DFW        86
         IAH         3
         LAS         3
Name: CANCELLED, dtype: int64

In [50]:
fg_unstack = fg.unstack('ORG_AIR', fill_value=0)

In [51]:
fg_unstack.head(3)

ORG_AIR  ATL  DEN  DFW  IAH  LAS  LAX  MSP  ORD  PHX  SFO
AIRLINE                                                  
AA         3    4   86    3    3   11    3   35    4    2
AS         0    0    0    0    0    0    0    0    0    0
B6         0    0    0    0    0    0    0    0    0    1

In [52]:
fp.equals(fg_unstack)

True

In [53]:
with_pivot = flights.pivot_table(
                        index=['AIRLINE', 'MONTH'],
                        columns=['ORG_AIR', 'CANCELLED'],
                        values=['DEP_DELAY', 'DIST'],
                        aggfunc=[np.sum, np.mean],
                        fill_value=0)

In [54]:
with_agg = (flights.groupby( 
                ['AIRLINE', 'MONTH', 'ORG_AIR', 'CANCELLED']) 
                ['DEP_DELAY', 'DIST'] 
               .agg(['sum', 'mean']) 
               .unstack(['ORG_AIR', 'CANCELLED'], fill_value=0) 
               .swaplevel(0, 1, axis='columns')).fillna(0)

<ipython-input-54-86a65d4d844f>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  with_agg = (flights.groupby(


In [55]:
with_pivot.head(4)

sum                                       ...  \
              DEP_DELAY                                       ...   
ORG_AIR             ATL     DEN      DFW     IAH      LAS     ...   
CANCELLED             0  1    0  1     0  1    0  1     0  1  ...   
AIRLINE MONTH                                                 ...   
AA      1           -13  0  113  0  4276 -3  117  0  1036  0  ...   
        2           -39  0   71  0  2662  0    8  0   -55  0  ...   
        3            -2  0   69  0  5692  0  109  0   326  0  ...   
        4             1  0  304  0  3518  0  104  0   790  0  ...   

                      mean                                                     \
                      DIST                                                      
ORG_AIR                LAX                  MSP               ORD               
CANCELLED                0       1            0    1            0           1   
AIRLINE MONTH                                                                   
AA      1      1678.037037  2475.0   809.000000  0.0  1068.876033    0.000000   
        2      1745.892308  1818.0  1008.000000  0.0  1193.782178  771.142857   
        3      1781.567568  1744.0   964.733333  0.0  1058.933333  802.000000   
        4      1850.923913     0.0   648.714286  0.0  1094.633094  943.600000   

                                                        
                                                        
ORG_AIR                PHX                 SFO          
CANCELLED                0      1            0       1  
AIRLINE MONTH                                           
AA      1      1167.666667    0.0  1860.166667     0.0  
        2      1311.461538  868.0  1337.916667  2586.0  
        3      1171.363636    0.0  1502.758621     0.0  
        4      1266.214286    0.0  1646.903226     0.0  

[4 rows x 80 columns]

In [56]:
with_agg.head(4)

sum                                                    \
              DEP_DELAY                                                     
ORG_AIR             ATL         DEN          DFW         IAH          LAS   
CANCELLED             0    1      0    1       0    1      0    1       0   
AIRLINE MONTH                                                               
AA      1         -13.0  0.0  113.0  0.0  4276.0 -3.0  117.0  0.0  1036.0   
        2         -39.0  0.0   71.0  0.0  2662.0  0.0    8.0  0.0   -55.0   
        3          -2.0  0.0   69.0  0.0  5692.0  0.0  109.0  0.0   326.0   
        4           1.0  0.0  304.0  0.0  3518.0  0.0  104.0  0.0   790.0   

                    ...         mean                                         \
                    ...         DIST                                          
ORG_AIR             ...          LAX                  MSP               ORD   
CANCELLED        1  ...            0       1            0    1            0   
AIRLINE MONTH       ...                                                       
AA      1      0.0  ...  1678.037037  2475.0   809.000000  0.0  1068.876033   
        2      0.0  ...  1745.892308  1818.0  1008.000000  0.0  1193.782178   
        3      0.0  ...  1781.567568  1744.0   964.733333  0.0  1058.933333   
        4      0.0  ...  1850.923913     0.0   648.714286  0.0  1094.633094   

                                                                    
                                                                    
ORG_AIR                            PHX                 SFO          
CANCELLED               1            0      1            0       1  
AIRLINE MONTH                                                       
AA      1        0.000000  1167.666667    0.0  1860.166667     0.0  
        2      771.142857  1311.461538  868.0  1337.916667  2586.0  
        3      802.000000  1171.363636    0.0  1502.758621     0.0  
        4      943.600000  1266.214286    0.0  1646.903226     0.0  

[4 rows x 80 columns]

In [57]:
with_pivot.equals(with_agg)

False

### Renaming axis levels with stack / unstack

In [58]:
college = pd.read_csv('data/college.csv')
cg = college.groupby(['STABBR', 'RELAFFIL']) \
                    ['UGDS', 'SATMTMID'] \
                .agg(['size', 'min', 'max']).head(6)

<ipython-input-58-c004a6809936>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  cg = college.groupby(['STABBR', 'RELAFFIL']) \


In [59]:
cg

UGDS                 SATMTMID              
                size    min      max     size    min    max
STABBR RELAFFIL                                            
AK     0           7  109.0  12865.0        7    NaN    NaN
       1           3   27.0    275.0        3  503.0  503.0
AL     0          72   12.0  29851.0       72  420.0  590.0
       1          24   13.0   3033.0       24  400.0  560.0
AR     0          68   18.0  21405.0       68  427.0  565.0
       1          18   20.0   4485.0       18  495.0  600.0

In [60]:
cg.index

MultiIndex([('AK', 0),
            ('AK', 1),
            ('AL', 0),
            ('AL', 1),
            ('AR', 0),
            ('AR', 1)],
           names=['STABBR', 'RELAFFIL'])

In [61]:
cg.columns

MultiIndex([(    'UGDS', 'size'),
            (    'UGDS',  'min'),
            (    'UGDS',  'max'),
            ('SATMTMID', 'size'),
            ('SATMTMID',  'min'),
            ('SATMTMID',  'max')],
           )

In [62]:
cg = cg.rename_axis(['AGG_COLS', 'AGG_FUNCS'], axis='columns')
cg

AGG_COLS        UGDS                 SATMTMID              
AGG_FUNCS       size    min      max     size    min    max
STABBR RELAFFIL                                            
AK     0           7  109.0  12865.0        7    NaN    NaN
       1           3   27.0    275.0        3  503.0  503.0
AL     0          72   12.0  29851.0       72  420.0  590.0
       1          24   13.0   3033.0       24  400.0  560.0
AR     0          68   18.0  21405.0       68  427.0  565.0
       1          18   20.0   4485.0       18  495.0  600.0

In [63]:
cg.stack('AGG_FUNCS').head()

AGG_COLS                      UGDS  SATMTMID
STABBR RELAFFIL AGG_FUNCS                   
AK     0        size           7.0       7.0
                min          109.0       NaN
                max        12865.0       NaN
       1        size           3.0       3.0
                min           27.0     503.0

In [64]:
cg.stack('AGG_FUNCS').swaplevel('AGG_FUNCS', 'STABBR',
                                    axis='index').head()

AGG_COLS                      UGDS  SATMTMID
AGG_FUNCS RELAFFIL STABBR                   
size      0        AK          7.0       7.0
min       0        AK        109.0       NaN
max       0        AK      12865.0       NaN
size      1        AK          3.0       3.0
min       1        AK         27.0     503.0

#### sorting on the stacked columns, by givng name

In [65]:
cg.stack('AGG_FUNCS') \
      .swaplevel('AGG_FUNCS', 'STABBR', axis='index') \
      .sort_index(level='RELAFFIL', axis='index') \
      .sort_index(level='AGG_COLS', axis='columns').head(6)

AGG_COLS                   SATMTMID     UGDS
AGG_FUNCS RELAFFIL STABBR                   
max       0        AK           NaN  12865.0
                   AL         590.0  29851.0
                   AR         565.0  21405.0
min       0        AK           NaN    109.0
                   AL         420.0     12.0
                   AR         427.0     18.0

In [66]:
cg.head()

AGG_COLS        UGDS                 SATMTMID              
AGG_FUNCS       size    min      max     size    min    max
STABBR RELAFFIL                                            
AK     0           7  109.0  12865.0        7    NaN    NaN
       1           3   27.0    275.0        3  503.0  503.0
AL     0          72   12.0  29851.0       72  420.0  590.0
       1          24   13.0   3033.0       24  400.0  560.0
AR     0          68   18.0  21405.0       68  427.0  565.0

In [67]:
cg.stack('AGG_FUNCS').unstack(['RELAFFIL', 'STABBR'])

AGG_COLS      UGDS                                          SATMTMID         \
RELAFFIL         0      1        0       1        0       1        0      1   
STABBR          AK     AK       AL      AL       AR      AR       AK     AK   
AGG_FUNCS                                                                     
size           7.0    3.0     72.0    24.0     68.0    18.0      7.0    3.0   
min          109.0   27.0     12.0    13.0     18.0    20.0      NaN  503.0   
max        12865.0  275.0  29851.0  3033.0  21405.0  4485.0      NaN  503.0   

AGG_COLS                               
RELAFFIL       0      1      0      1  
STABBR        AL     AL     AR     AR  
AGG_FUNCS                              
size        72.0   24.0   68.0   18.0  
min        420.0  400.0  427.0  495.0  
max        590.0  560.0  565.0  600.0

#### stacking all available columns

In [68]:
cg.stack(['AGG_FUNCS', 'AGG_COLS']).head(12)

STABBR  RELAFFIL  AGG_FUNCS  AGG_COLS
AK      0         size       UGDS            7.0
                             SATMTMID        7.0
                  min        UGDS          109.0
                  max        UGDS        12865.0
        1         size       UGDS            3.0
                             SATMTMID        3.0
                  min        UGDS           27.0
                             SATMTMID      503.0
                  max        UGDS          275.0
                             SATMTMID      503.0
AL      0         size       UGDS           72.0
                             SATMTMID       72.0
dtype: float64

In [69]:
cg.head()

AGG_COLS        UGDS                 SATMTMID              
AGG_FUNCS       size    min      max     size    min    max
STABBR RELAFFIL                                            
AK     0           7  109.0  12865.0        7    NaN    NaN
       1           3   27.0    275.0        3  503.0  503.0
AL     0          72   12.0  29851.0       72  420.0  590.0
       1          24   13.0   3033.0       24  400.0  560.0
AR     0          68   18.0  21405.0       68  427.0  565.0

#### to remove the generared index and column values from agg/stack

In [70]:
cg.rename_axis([None, None], axis='index') \
      .rename_axis([None, None], axis='columns')

UGDS                 SATMTMID              
     size    min      max     size    min    max
AK 0    7  109.0  12865.0        7    NaN    NaN
   1    3   27.0    275.0        3  503.0  503.0
AL 0   72   12.0  29851.0       72  420.0  590.0
   1   24   13.0   3033.0       24  400.0  560.0
AR 0   68   18.0  21405.0       68  427.0  565.0
   1   18   20.0   4485.0       18  495.0  600.0

### Dealing with - Multiple variables stored as one column values

In [71]:
weightlifting = pd.read_csv('data/weightlifting_men.csv')
weightlifting

Weight Category  M35 35-39  M40 40-44  M45 45-49  M50 50-54  M55 55-59  \
0              56        137        130        125        115        102   
1              62        152        145        137        127        112   
2              69        167        160        150        140        125   
3              77        182        172        165        150        135   
4              85        192        182        175        160        142   
5              94        202        192        182        167        150   
6             105        210        200        190        175        157   
7            105+        217        207        197        182        165   

   M60 60-64  M65 65-69  M70 70-74  M75 75-79  M80 80+  
0         92         80         67         62       55  
1        102         90         75         67       57  
2        112         97         82         75       60  
3        122        107         90         82       65  
4        130        112         95         87       70  
5        137        120        100         90       75  
6        142        122        102         95       80  
7        150        127        107        100       85

In [72]:
wl_melt = weightlifting.melt(id_vars='Weight Category', 
                             var_name='sex_age', 
                             value_name='Qual Total')
wl_melt.head(15)

Weight Category    sex_age  Qual Total
0               56  M35 35-39         137
1               62  M35 35-39         152
2               69  M35 35-39         167
3               77  M35 35-39         182
4               85  M35 35-39         192
5               94  M35 35-39         202
6              105  M35 35-39         210
7             105+  M35 35-39         217
8               56  M40 40-44         130
9               62  M40 40-44         145
10              69  M40 40-44         160
11              77  M40 40-44         172
12              85  M40 40-44         182
13              94  M40 40-44         192
14             105  M40 40-44         200

#### Age and sex are combined as M35, we can split this

In [73]:
sex_age = wl_melt['sex_age'].str.split(expand=True)
sex_age.head()

0      1
0  M35  35-39
1  M35  35-39
2  M35  35-39
3  M35  35-39
4  M35  35-39

In [74]:
sex_age.columns = ['Sex', 'Age Group']
sex_age.head()

Sex Age Group
0  M35     35-39
1  M35     35-39
2  M35     35-39
3  M35     35-39
4  M35     35-39

In [75]:
sex_age['Sex'] = sex_age['Sex'].str[0]
sex_age.head()

Sex Age Group
0   M     35-39
1   M     35-39
2   M     35-39
3   M     35-39
4   M     35-39

In [76]:
wl_cat_total = wl_melt[['Weight Category', 'Qual Total']]
wl_tidy = pd.concat([sex_age, wl_cat_total], axis='columns') # combining on sme index.
wl_tidy.head()

Sex Age Group Weight Category  Qual Total
0   M     35-39              56         137
1   M     35-39              62         152
2   M     35-39              69         167
3   M     35-39              77         182
4   M     35-39              85         192

#### Using regular expression

In [77]:
wl_melt.head(4)

Weight Category    sex_age  Qual Total
0              56  M35 35-39         137
1              62  M35 35-39         152
2              69  M35 35-39         167
3              77  M35 35-39         182

In [78]:
age_group = wl_melt.sex_age.str.extract('(\d{2}[-+](?:\d{2})?)', expand=False)

In [79]:
sex = wl_melt.sex_age.str[0]

In [80]:
age_group.shape, sex.shape 

((80,), (80,))

In [81]:
new_cols = {'Sex':sex, 'Age Group': age_group}

In [82]:
wl_tidy2 = wl_melt.assign(**new_cols).drop('sex_age',axis='columns')

In [83]:
wl_tidy2.head(4)

Weight Category  Qual Total Sex Age Group
0              56         137   M     35-39
1              62         152   M     35-39
2              69         167   M     35-39
3              77         182   M     35-39

### Multiple variables stored as column names

In [84]:
inspections = pd.read_csv('data/restaurant_inspections.csv', parse_dates=['Date'])
inspections.head(9)

Name       Date         Info  \
0  E & E Grill House 2017-08-08      Borough   
1  E & E Grill House 2017-08-08      Cuisine   
2  E & E Grill House 2017-08-08  Description   
3  E & E Grill House 2017-08-08        Grade   
4  E & E Grill House 2017-08-08        Score   
5        PIZZA WAGON 2017-04-12      Borough   
6        PIZZA WAGON 2017-04-12      Cuisine   
7        PIZZA WAGON 2017-04-12  Description   
8        PIZZA WAGON 2017-04-12        Grade   

                                               Value  
0                                          MANHATTAN  
1                                           American  
2  Non-food contact surface improperly constructe...  
3                                                  A  
4                                                9.0  
5                                           BROOKLYN  
6                                              Pizza  
7  Food contact surface not properly washed, rins...  
8                                                  A

In [85]:
inspections.columns

Index(['Name', 'Date', 'Info', 'Value'], dtype='object')

#### The Info columns looks like variables, it can be moved to columns

In [86]:
inspections.index

RangeIndex(start=0, stop=500, step=1)

In [87]:
inspections.set_index(['Name', 'Date', 'Info']).head(10)

Value
Name              Date       Info                                                          
E & E Grill House 2017-08-08 Borough                                              MANHATTAN
                             Cuisine                                               American
                             Description  Non-food contact surface improperly constructe...
                             Grade                                                        A
                             Score                                                      9.0
PIZZA WAGON       2017-04-12 Borough                                               BROOKLYN
                             Cuisine                                                  Pizza
                             Description  Food contact surface not properly washed, rins...
                             Grade                                                        A
                             Score                                                     10.0

In [88]:
inspections.set_index(['Name', 'Date', 'Info']).unstack('Info').head()

Value  \
Info                                              Borough   
Name                                Date                    
3 STAR JUICE CENTER                 2017-05-10   BROOKLYN   
A & L PIZZA RESTAURANT              2017-08-22   BROOKLYN   
AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25   BROOKLYN   
ANTOJITOS DELI FOOD                 2017-06-01   BROOKLYN   
BANGIA                              2017-06-16  MANHATTAN   

                                                                                                   \
Info                                                                                      Cuisine   
Name                                Date                                                            
3 STAR JUICE CENTER                 2017-05-10                     Juice, Smoothies, Fruit Salads   
A & L PIZZA RESTAURANT              2017-08-22                                              Pizza   
AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25                                            Turkish   
ANTOJITOS DELI FOOD                 2017-06-01  Latin (Cuban, Dominican, Puerto Rican, South &...   
BANGIA                              2017-06-16                                             Korean   

                                                                                                   \
Info                                                                                  Description   
Name                                Date                                                            
3 STAR JUICE CENTER                 2017-05-10  Facility not vermin proof. Harborage or condit...   
A & L PIZZA RESTAURANT              2017-08-22  Facility not vermin proof. Harborage or condit...   
AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25  Plumbing not properly installed or maintained;...   
ANTOJITOS DELI FOOD                 2017-06-01  Live roaches present in facility's food and/or...   
BANGIA                              2017-06-16  Covered garbage receptacle not provided or ina...   

                                                            
Info                                           Grade Score  
Name                                Date                    
3 STAR JUICE CENTER                 2017-05-10     A  12.0  
A & L PIZZA RESTAURANT              2017-08-22     A   9.0  
AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25     A  13.0  
ANTOJITOS DELI FOOD                 2017-06-01     A  10.0  
BANGIA                              2017-06-16     A   9.0

In [89]:
inspections.set_index(['Name', 'Date', 'Info']).unstack('Info').columns

MultiIndex([('Value',     'Borough'),
            ('Value',     'Cuisine'),
            ('Value', 'Description'),
            ('Value',       'Grade'),
            ('Value',       'Score')],
           names=[None, 'Info'])

In [90]:
inspections.set_index(['Name', 'Date', 'Info']).unstack('Info').index[:6]

MultiIndex([(                '3 STAR JUICE CENTER', '2017-05-10'),
            (             'A & L PIZZA RESTAURANT', '2017-08-22'),
            ('AKSARAY TURKISH CAFE AND RESTAURANT', '2017-07-25'),
            (                'ANTOJITOS DELI FOOD', '2017-06-01'),
            (                             'BANGIA', '2017-06-16'),
            (                    'BANGKOK CUISINE', '2017-07-19')],
           names=['Name', 'Date'])

In [91]:
insp_tidy = inspections.set_index(['Name','Date', 'Info']) \
                           .unstack('Info') \
                           .reset_index(col_level=-1)
insp_tidy.head(10)

Value  \
Info                                 Name       Date    Borough   
0                     3 STAR JUICE CENTER 2017-05-10   BROOKLYN   
1                  A & L PIZZA RESTAURANT 2017-08-22   BROOKLYN   
2     AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25   BROOKLYN   
3                     ANTOJITOS DELI FOOD 2017-06-01   BROOKLYN   
4                                  BANGIA 2017-06-16  MANHATTAN   
5                         BANGKOK CUISINE 2017-07-19  MANHATTAN   
6                                   BASIL 2017-05-03   BROOKLYN   
7                              BEIT JEDDO 2017-03-23   BROOKLYN   
8                  BIG FLEISHIG'S EXPRESS 2017-02-22   BROOKLYN   
9                    BLOSSOM  ON COLUMBUS 2017-01-25  MANHATTAN   

                                                         \
Info                                            Cuisine   
0                        Juice, Smoothies, Fruit Salads   
1                                                 Pizza   
2                                               Turkish   
3     Latin (Cuban, Dominican, Puerto Rican, South &...   
4                                                Korean   
5                                                  Thai   
6                                         Jewish/Kosher   
7                                        Middle Eastern   
8                                         Jewish/Kosher   
9                                              American   

                                                                     
Info                                        Description Grade Score  
0     Facility not vermin proof. Harborage or condit...     A  12.0  
1     Facility not vermin proof. Harborage or condit...     A   9.0  
2     Plumbing not properly installed or maintained;...     A  13.0  
3     Live roaches present in facility's food and/or...     A  10.0  
4     Covered garbage receptacle not provided or ina...     A   9.0  
5     Non-food contact surface improperly constructe...     A  13.0  
6     Cold food item held above 41Âº F (smoked fish ...     A  13.0  
7                          Thawing procedures improper.     A  13.0  
8     Single service item reused, improperly stored,...     A  12.0  
9     Food contact surface not properly washed, rins...     A  10.0

In [92]:
insp_tidy.columns = insp_tidy.columns.droplevel(0) \
                                         .rename(None)
insp_tidy.head(10)

Name       Date    Borough  \
0                  3 STAR JUICE CENTER 2017-05-10   BROOKLYN   
1               A & L PIZZA RESTAURANT 2017-08-22   BROOKLYN   
2  AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25   BROOKLYN   
3                  ANTOJITOS DELI FOOD 2017-06-01   BROOKLYN   
4                               BANGIA 2017-06-16  MANHATTAN   
5                      BANGKOK CUISINE 2017-07-19  MANHATTAN   
6                                BASIL 2017-05-03   BROOKLYN   
7                           BEIT JEDDO 2017-03-23   BROOKLYN   
8               BIG FLEISHIG'S EXPRESS 2017-02-22   BROOKLYN   
9                 BLOSSOM  ON COLUMBUS 2017-01-25  MANHATTAN   

                                             Cuisine  \
0                     Juice, Smoothies, Fruit Salads   
1                                              Pizza   
2                                            Turkish   
3  Latin (Cuban, Dominican, Puerto Rican, South &...   
4                                             Korean   
5                                               Thai   
6                                      Jewish/Kosher   
7                                     Middle Eastern   
8                                      Jewish/Kosher   
9                                           American   

                                         Description Grade Score  
0  Facility not vermin proof. Harborage or condit...     A  12.0  
1  Facility not vermin proof. Harborage or condit...     A   9.0  
2  Plumbing not properly installed or maintained;...     A  13.0  
3  Live roaches present in facility's food and/or...     A  10.0  
4  Covered garbage receptacle not provided or ina...     A   9.0  
5  Non-food contact surface improperly constructe...     A  13.0  
6  Cold food item held above 41Âº F (smoked fish ...     A  13.0  
7                       Thawing procedures improper.     A  13.0  
8  Single service item reused, improperly stored,...     A  12.0  
9  Food contact surface not properly washed, rins...     A  10.0

In [93]:
inspections.set_index(['Name','Date', 'Info']) \
               .squeeze() \
               .unstack('Info') \
               .reset_index() \
               .rename_axis(None, axis='columns')

Name       Date        Borough  \
0                   3 STAR JUICE CENTER 2017-05-10       BROOKLYN   
1                A & L PIZZA RESTAURANT 2017-08-22       BROOKLYN   
2   AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25       BROOKLYN   
3                   ANTOJITOS DELI FOOD 2017-06-01       BROOKLYN   
4                                BANGIA 2017-06-16      MANHATTAN   
..                                  ...        ...            ...   
95                      VALL'S PIZZERIA 2017-03-15  STATEN ISLAND   
96                            VIP GRILL 2017-06-12       BROOKLYN   
97                              WAHIZZA 2017-04-13      MANHATTAN   
98                    WANG MANDOO HOUSE 2017-08-29         QUEENS   
99                     XIAOYAN YABO INC 2017-08-29         QUEENS   

                                              Cuisine  \
0                      Juice, Smoothies, Fruit Salads   
1                                               Pizza   
2                                             Turkish   
3   Latin (Cuban, Dominican, Puerto Rican, South &...   
4                                              Korean   
..                                                ...   
95                                      Pizza/Italian   
96                                      Jewish/Kosher   
97                                              Pizza   
98                                             Korean   
99                                             Korean   

                                          Description Grade Score  
0   Facility not vermin proof. Harborage or condit...     A  12.0  
1   Facility not vermin proof. Harborage or condit...     A   9.0  
2   Plumbing not properly installed or maintained;...     A  13.0  
3   Live roaches present in facility's food and/or...     A  10.0  
4   Covered garbage receptacle not provided or ina...     A   9.0  
..                                                ...   ...   ...  
95  Wiping cloths soiled or not stored in sanitizi...     A   9.0  
96        Hot food item not held at or above 140Âº F.     A  10.0  
97  No facilities available to wash, rinse and san...     A  10.0  
98  Accurate thermometer not provided in refrigera...     A  12.0  
99  Evidence of mice or live mice present in facil...     Z  49.0  

[100 rows x 7 columns]

#### Using pivot table and aggfunc to choose 'first', we can achieve the same as above.

In [94]:
inspections.pivot_table(index=['Name', 'Date'], 
                            columns='Info', 
                            values='Value', 
                            aggfunc='first') \
               .reset_index() \
               .rename_axis(None, axis='columns')

Name       Date        Borough  \
0                   3 STAR JUICE CENTER 2017-05-10       BROOKLYN   
1                A & L PIZZA RESTAURANT 2017-08-22       BROOKLYN   
2   AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25       BROOKLYN   
3                   ANTOJITOS DELI FOOD 2017-06-01       BROOKLYN   
4                                BANGIA 2017-06-16      MANHATTAN   
..                                  ...        ...            ...   
95                      VALL'S PIZZERIA 2017-03-15  STATEN ISLAND   
96                            VIP GRILL 2017-06-12       BROOKLYN   
97                              WAHIZZA 2017-04-13      MANHATTAN   
98                    WANG MANDOO HOUSE 2017-08-29         QUEENS   
99                     XIAOYAN YABO INC 2017-08-29         QUEENS   

                                              Cuisine  \
0                      Juice, Smoothies, Fruit Salads   
1                                               Pizza   
2                                             Turkish   
3   Latin (Cuban, Dominican, Puerto Rican, South &...   
4                                              Korean   
..                                                ...   
95                                      Pizza/Italian   
96                                      Jewish/Kosher   
97                                              Pizza   
98                                             Korean   
99                                             Korean   

                                          Description Grade Score  
0   Facility not vermin proof. Harborage or condit...     A  12.0  
1   Facility not vermin proof. Harborage or condit...     A   9.0  
2   Plumbing not properly installed or maintained;...     A  13.0  
3   Live roaches present in facility's food and/or...     A  10.0  
4   Covered garbage receptacle not provided or ina...     A   9.0  
..                                                ...   ...   ...  
95  Wiping cloths soiled or not stored in sanitizi...     A   9.0  
96        Hot food item not held at or above 140Âº F.     A  10.0  
97  No facilities available to wash, rinse and san...     A  10.0  
98  Accurate thermometer not provided in refrigera...     A  12.0  
99  Evidence of mice or live mice present in facil...     Z  49.0  

[100 rows x 7 columns]

### More than one value stored in same cell

In [95]:
cities = pd.read_csv('data/texas_cities.csv')
cities

City             Geolocation
0  Houston  29.7604° N, 95.3698° W
1   Dallas  32.7767° N, 96.7970° W
2   Austin  30.2672° N, 97.7431° W

#### Geo location columns has 4 values ( lan, long, directions), 
#### can be used as four columns for better clarity

In [96]:
geolocations = cities.Geolocation.str.split(pat='. ', expand=True)
geolocations

0  1        2  3
0  29.7604  N  95.3698  W
1  32.7767  N  96.7970  W
2  30.2672  N  97.7431  W

In [97]:
geolocations.columns = ['latitude', 'latitude direction',
                            'longitude', 'longitude direction']
geolocations

latitude latitude direction longitude longitude direction
0  29.7604                  N   95.3698                   W
1  32.7767                  N   96.7970                   W
2  30.2672                  N   97.7431                   W

In [98]:
geolocations.dtypes

latitude               object
latitude direction     object
longitude              object
longitude direction    object
dtype: object

In [99]:
geolocations = geolocations.astype({'latitude':'float',
                                        'longitude':'float'})
geolocations.dtypes

latitude               float64
latitude direction      object
longitude              float64
longitude direction     object
dtype: object

In [100]:
cities_tidy = pd.concat([cities['City'], geolocations], axis='columns')

In [101]:
cities_tidy

City  latitude latitude direction  longitude longitude direction
0  Houston   29.7604                  N    95.3698                   W
1   Dallas   32.7767                  N    96.7970                   W
2   Austin   30.2672                  N    97.7431                   W

### Variables stored in both column names and values (vertical and horizontal)

In [102]:
sensors = pd.read_csv('data/sensors.csv')
sensors

Group     Property  2012  2013  2014  2015  2016
0     A     Pressure   928   873   814   973   870
1     A  Temperature  1026  1038  1009  1036  1042
2     A         Flow   819   806   861   882   856
3     B     Pressure   817   877   914   806   942
4     B  Temperature  1008  1041  1009  1002  1013
5     B         Flow   887   899   837   824   873

In [103]:
sensor_melted = sensors.melt(id_vars=['Group', 'Property'], var_name='Year')
sensor_melted.head(6)

Group     Property  Year  value
0     A     Pressure  2012    928
1     A  Temperature  2012   1026
2     A         Flow  2012    819
3     B     Pressure  2012    817
4     B  Temperature  2012   1008
5     B         Flow  2012    887

In [104]:
sensor_melted_pivot = (sensor_melted.pivot_table(index=['Group', 'Year'],
                                                columns='Property',
                                                values='value').reset_index()
                                    .rename_axis(None, axis='columns'))
                                    

In [105]:
sensor_melted_pivot.head(6)

Group  Year  Flow  Pressure  Temperature
0     A  2012   819       928         1026
1     A  2013   806       873         1038
2     A  2014   861       814         1009
3     A  2015   882       973         1036
4     A  2016   856       870         1042
5     B  2012   887       817         1008

### multiple observations stored in the same table

In [106]:
movie = pd.read_csv('data/movie_altered.csv')
movie.head()

title rating    year  duration  \
0                                      Avatar  PG-13  2009.0     178.0   
1    Pirates of the Caribbean: At World's End  PG-13  2007.0     169.0   
2                                     Spectre  PG-13  2015.0     148.0   
3                       The Dark Knight Rises  PG-13  2012.0     164.0   
4  Star Wars: Episode VII - The Force Awakens    NaN     NaN       NaN   

          director_1  director_fb_likes_1          actor_1           actor_2  \
0      James Cameron                  0.0      CCH Pounder  Joel David Moore   
1     Gore Verbinski                563.0      Johnny Depp     Orlando Bloom   
2         Sam Mendes                  0.0  Christoph Waltz      Rory Kinnear   
3  Christopher Nolan              22000.0        Tom Hardy    Christian Bale   
4        Doug Walker                131.0      Doug Walker        Rob Walker   

                actor_3  actor_fb_likes_1  actor_fb_likes_2  actor_fb_likes_3  
0             Wes Studi            1000.0             936.0             855.0  
1        Jack Davenport           40000.0            5000.0            1000.0  
2      Stephanie Sigman           11000.0             393.0             161.0  
3  Joseph Gordon-Levitt           27000.0           23000.0           23000.0  
4                   NaN             131.0              12.0               NaN

In [107]:
movie.insert(0, 'id', np.arange(len(movie)))
movie.head()

id                                       title rating    year  duration  \
0   0                                      Avatar  PG-13  2009.0     178.0   
1   1    Pirates of the Caribbean: At World's End  PG-13  2007.0     169.0   
2   2                                     Spectre  PG-13  2015.0     148.0   
3   3                       The Dark Knight Rises  PG-13  2012.0     164.0   
4   4  Star Wars: Episode VII - The Force Awakens    NaN     NaN       NaN   

          director_1  director_fb_likes_1          actor_1           actor_2  \
0      James Cameron                  0.0      CCH Pounder  Joel David Moore   
1     Gore Verbinski                563.0      Johnny Depp     Orlando Bloom   
2         Sam Mendes                  0.0  Christoph Waltz      Rory Kinnear   
3  Christopher Nolan              22000.0        Tom Hardy    Christian Bale   
4        Doug Walker                131.0      Doug Walker        Rob Walker   

                actor_3  actor_fb_likes_1  actor_fb_likes_2  actor_fb_likes_3  
0             Wes Studi            1000.0             936.0             855.0  
1        Jack Davenport           40000.0            5000.0            1000.0  
2      Stephanie Sigman           11000.0             393.0             161.0  
3  Joseph Gordon-Levitt           27000.0           23000.0           23000.0  
4                   NaN             131.0              12.0               NaN

In [108]:
stubnames = ['director', 'director_fb_likes','actor', 'actor_fb_likes']

In [109]:
movie_long = pd.wide_to_long(movie,
                            stubnames=stubnames,
                             i='id',
                             j='num',
                             sep='_').reset_index()

In [110]:
movie_long['num'] = movie_long['num'].astype(int)
movie_long.head(9)

id  num                                     title    year  duration rating  \
0   0    1                                    Avatar  2009.0     178.0  PG-13   
1   0    2                                    Avatar  2009.0     178.0  PG-13   
2   0    3                                    Avatar  2009.0     178.0  PG-13   
3   1    1  Pirates of the Caribbean: At World's End  2007.0     169.0  PG-13   
4   1    2  Pirates of the Caribbean: At World's End  2007.0     169.0  PG-13   
5   1    3  Pirates of the Caribbean: At World's End  2007.0     169.0  PG-13   
6   2    1                                   Spectre  2015.0     148.0  PG-13   
7   2    2                                   Spectre  2015.0     148.0  PG-13   
8   2    3                                   Spectre  2015.0     148.0  PG-13   

         director  director_fb_likes             actor  actor_fb_likes  
0   James Cameron                0.0       CCH Pounder          1000.0  
1             NaN                NaN  Joel David Moore           936.0  
2             NaN                NaN         Wes Studi           855.0  
3  Gore Verbinski              563.0       Johnny Depp         40000.0  
4             NaN                NaN     Orlando Bloom          5000.0  
5             NaN                NaN    Jack Davenport          1000.0  
6      Sam Mendes                0.0   Christoph Waltz         11000.0  
7             NaN                NaN      Rory Kinnear           393.0  
8             NaN                NaN  Stephanie Sigman           161.0

In [111]:
movie_long.index

RangeIndex(start=0, stop=14748, step=1)

In [112]:
movie_table = movie_long[['id', 'title', 'year', 'duration', 'rating']]
director_table = movie_long[['id', 'num','director', 'director_fb_likes']]
actor_table = movie_long[['id', 'num', 'actor', 'actor_fb_likes']]

In [113]:
movie_table.head()
director_table.head()
actor_table.head()

id                                     title    year  duration rating
0   0                                    Avatar  2009.0     178.0  PG-13
1   0                                    Avatar  2009.0     178.0  PG-13
2   0                                    Avatar  2009.0     178.0  PG-13
3   1  Pirates of the Caribbean: At World's End  2007.0     169.0  PG-13
4   1  Pirates of the Caribbean: At World's End  2007.0     169.0  PG-13

id  num        director  director_fb_likes
0   0    1   James Cameron                0.0
1   0    2             NaN                NaN
2   0    3             NaN                NaN
3   1    1  Gore Verbinski              563.0
4   1    2             NaN                NaN

id  num             actor  actor_fb_likes
0   0    1       CCH Pounder          1000.0
1   0    2  Joel David Moore           936.0
2   0    3         Wes Studi           855.0
3   1    1       Johnny Depp         40000.0
4   1    2     Orlando Bloom          5000.0

In [114]:
movie_entity = movie_table.drop_duplicates().reset_index(drop=True)
director_entity = director_table.dropna().reset_index(drop=True)
actor_entity = actor_table.dropna().reset_index(drop=True)

In [115]:
movie_entity.head()
director_entity.head()
actor_entity.head()

id                                       title    year  duration rating
0   0                                      Avatar  2009.0     178.0  PG-13
1   1    Pirates of the Caribbean: At World's End  2007.0     169.0  PG-13
2   2                                     Spectre  2015.0     148.0  PG-13
3   3                       The Dark Knight Rises  2012.0     164.0  PG-13
4   4  Star Wars: Episode VII - The Force Awakens     NaN       NaN    NaN

id  num           director  director_fb_likes
0   0    1      James Cameron                0.0
1   1    1     Gore Verbinski              563.0
2   2    1         Sam Mendes                0.0
3   3    1  Christopher Nolan            22000.0
4   4    1        Doug Walker              131.0

id  num             actor  actor_fb_likes
0   0    1       CCH Pounder          1000.0
1   0    2  Joel David Moore           936.0
2   0    3         Wes Studi           855.0
3   1    1       Johnny Depp         40000.0
4   1    2     Orlando Bloom          5000.0

In [116]:
movie.memory_usage(deep=True).sum()

2300682

In [117]:
(movie_entity.memory_usage(deep=True).sum() + 
director_entity.memory_usage(deep=True).sum() +
actor_entity.memory_usage(deep=True).sum())

2607442

In [118]:
director_cat = pd.Categorical(director_entity['director'])
director_entity.insert(1, 'director_id', director_cat.codes)

In [119]:
actor_cat = pd.Categorical(actor_entity['actor'])
actor_entity.insert(1, 'actor_id', actor_cat.codes)

In [120]:
director_entity.head()
actor_entity.head()

id  director_id  num           director  director_fb_likes
0   0          922    1      James Cameron                0.0
1   1          794    1     Gore Verbinski              563.0
2   2         2020    1         Sam Mendes                0.0
3   3          373    1  Christopher Nolan            22000.0
4   4          600    1        Doug Walker              131.0

id  actor_id  num             actor  actor_fb_likes
0   0       824    1       CCH Pounder          1000.0
1   0      2867    2  Joel David Moore           936.0
2   0      6099    3         Wes Studi           855.0
3   1      2971    1       Johnny Depp         40000.0
4   1      4536    2     Orlando Bloom          5000.0

#### creating intemediatory table, for many-many

In [121]:
director_associative = director_entity[['id', 'director_id','num']]
dcols = ['director_id', 'director', 'director_fb_likes']

In [122]:
director_unique = director_entity[dcols].drop_duplicates().reset_index(drop=True)

In [123]:
director_unique.head()

director_id           director  director_fb_likes
0          922      James Cameron                0.0
1          794     Gore Verbinski              563.0
2         2020         Sam Mendes                0.0
3          373  Christopher Nolan            22000.0
4          600        Doug Walker              131.0

In [124]:
director_associative.head(6)

id  director_id  num
0   0          922    1
1   1          794    1
2   2         2020    1
3   3          373    1
4   4          600    1
5   5           99    1

In [125]:
actor_associative = actor_entity[['id', 'actor_id', 'num']]
acols = ['actor_id', 'actor', 'actor_fb_likes']
actor_unique = actor_entity[acols].drop_duplicates().reset_index(drop=True)

In [126]:
actor_associative.head(6)

id  actor_id  num
0   0       824    1
1   0      2867    2
2   0      6099    3
3   1      2971    1
4   1      4536    2
5   1      2403    3

In [127]:
(movie_entity.memory_usage(deep=True).sum() + 
    director_associative.memory_usage(deep=True).sum() + 
    director_unique.memory_usage(deep=True).sum() + 
    actor_associative.memory_usage(deep=True).sum() + 
    actor_unique.memory_usage(deep=True).sum())

1816042

In [128]:
movie.memory_usage(deep=True).sum()

2307693

#### rejoining back all the tables, to get the original table

In [129]:
# actors

actors = actor_associative.merge(actor_unique, on='actor_id') \
                              .drop('actor_id', 1) \
                              .pivot_table(index='id', columns='num', aggfunc='first')

In [130]:
actors.columns = actors.columns.get_level_values(0) + '_' + \
                     actors.columns.get_level_values(1).astype(str)

In [131]:
# directors

In [132]:
directors = director_associative.merge(director_unique,
                                           on='director_id') \
                                    .drop('director_id', 1) \
                                    .pivot_table(index='id',
                                                 columns='num',
                                                 aggfunc='first')

In [133]:
directors.columns = directors.columns.get_level_values(0) + '_' + \
                        directors.columns.get_level_values(1) \
                                         .astype(str)

In [134]:
actors.head()

actor_1           actor_2               actor_3  actor_fb_likes_1  \
id                                                                              
0       CCH Pounder  Joel David Moore             Wes Studi            1000.0   
1       Johnny Depp     Orlando Bloom        Jack Davenport           40000.0   
2   Christoph Waltz      Rory Kinnear      Stephanie Sigman           11000.0   
3         Tom Hardy    Christian Bale  Joseph Gordon-Levitt           27000.0   
4       Doug Walker        Rob Walker                   NaN             131.0   

    actor_fb_likes_2  actor_fb_likes_3  
id                                      
0              936.0             855.0  
1             5000.0            1000.0  
2              393.0             161.0  
3            23000.0           23000.0  
4               12.0               NaN

In [135]:
directors.head()

director_1  director_fb_likes_1
id                                        
0       James Cameron                  0.0
1      Gore Verbinski                563.0
2          Sam Mendes                  0.0
3   Christopher Nolan              22000.0
4         Doug Walker                131.0

In [136]:
movie_recreated = movie_entity.merge(directors.reset_index(), on='id', how='left') \
                        .merge(actors.reset_index(), on='id', how='left')

In [137]:
movie_recreated.memory_usage(deep=True).sum()

2346893

In [138]:
movie.memory_usage(deep=True).sum()

2307693

In [139]:
movie_recreated.equals(movie)

False